# Business Question

use machine learning to create a model that predicts which passengers survived the Titanic shipwreck. we’ll gain access to two similar datasets that include passenger information like name, age, gender, socio-economic class, etc. One dataset is titled `train.csv` and the other is titled `test.csv`.

Train.csv will contain the details of a subset of the passengers on board (891 to be exact) and importantly, will reveal whether they survived or not, also known as the “ground truth”.

The `test.csv` dataset contains similar information but does not disclose the “ground truth” for each passenger. It’s your job to predict these outcomes

# importing

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data Understanding

In [6]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
combine = [train_df, test_df]

In [7]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
train_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [9]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [10]:
print(test_df.shape)
print(train_df.shape)

(418, 11)
(891, 12)


In [11]:
train_df.info()
print('_'*50,"\n")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
__________________________________________________ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0

In [12]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
train_df.describe(include="O")

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [14]:
for col in train_df.columns:
    print(col,":",train_df[col].nunique())
    print(train_df[col].value_counts().nlargest(5))
    print('\n' + '*' * 20 + '\n')

PassengerId : 891
1      1
599    1
588    1
589    1
590    1
Name: PassengerId, dtype: int64

********************

Survived : 2
0    549
1    342
Name: Survived, dtype: int64

********************

Pclass : 3
3    491
1    216
2    184
Name: Pclass, dtype: int64

********************

Name : 891
Braund, Mr. Owen Harris             1
Boulos, Mr. Hanna                   1
Frolicher-Stehli, Mr. Maxmillian    1
Gilinski, Mr. Eliezer               1
Murdlin, Mr. Joseph                 1
Name: Name, dtype: int64

********************

Sex : 2
male      577
female    314
Name: Sex, dtype: int64

********************

Age : 88
24.0    30
22.0    27
18.0    26
19.0    25
28.0    25
Name: Age, dtype: int64

********************

SibSp : 7
0    608
1    209
2     28
4     18
3     16
Name: SibSp, dtype: int64

********************

Parch : 7
0    678
1    118
2     80
5      5
3      5
Name: Parch, dtype: int64

********************

Ticket : 681
347082      7
CA. 2343    7
1601        7
31012

# Data Cleaning

In [15]:
"""There's 21 passengers have cabin num C23 C25 C27 and 24 passengers have B96 B98 cabin num and thus is incredible.
so we will drop this data."""
train_df=train_df[train_df['Cabin']!='C23 C25 C27']
train_df=train_df[train_df['Cabin']!='B96 B98']

In [16]:
train_df.Cabin.value_counts().nlargest(5)

G6         4
F2         3
C22 C26    3
D          3
F33        3
Name: Cabin, dtype: int64

In [17]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
train_df.Age.interpolate(method ='linear',inplace=True)

In [19]:
train_df.Age.isnull().sum()

0

In [20]:
train_df.dropna(subset=['Embarked'],axis=0,how ='all',inplace=True)

In [21]:
train_df.Cabin.fillna(method ='bfill', inplace = True)
train_df.Cabin.fillna(method ='ffill', inplace = True)

In [22]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [23]:
train_df.duplicated().sum()

0

> data doesn't contain duplicated data

In [24]:
# converting float data into integer
train_df.Age=train_df.Age.astype(int)
train_df.Fare=train_df.Fare.astype(int)

In [25]:
train_df.Ticket

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 881, dtype: object

In [26]:
# Handling ticket number
if any(item.startswith("[A-Za-z]") for item in train_df.Ticket):
    train_df=train_df[train_df['Ticket']!=item]

In [46]:
arr=(train_df.Ticket.str.split(" ", n = 1, expand = False)).to_list()

In [77]:
new_arr=[]
for i in arr:
    if len(i)>1:
#         print(i)
        new_arr.append(((i[1])))
    else:
        new_arr.append(i[0])
# np.array(arr[0][1])

In [78]:
new_arr

['21171',
 '17599',
 '3101282',
 '113803',
 '373450',
 '330877',
 '17463',
 '349909',
 '347742',
 '237736',
 '9549',
 '113783',
 '2151',
 '347082',
 '350406',
 '248706',
 '382652',
 '244373',
 '345763',
 '2649',
 '239865',
 '248698',
 '330923',
 '113788',
 '349909',
 '347077',
 '2631',
 '330959',
 '349216',
 '17601',
 '17569',
 '335677',
 '24579',
 '17604',
 '113789',
 '2677',
 '2152',
 '345764',
 '2651',
 '7546',
 '11668',
 '349253',
 '2123',
 '330958',
 '23567',
 '370371',
 '14311',
 '2662',
 '349237',
 '3101295',
 '39886',
 '17572',
 '2926',
 '113509',
 '19947',
 '31026',
 '2697',
 '34651',
 '2144',
 '2669',
 '36973',
 '347088',
 '17605',
 '2661',
 '29395',
 '3464',
 '3101281',
 '315151',
 '33111',
 '2144',
 '14879',
 '2680',
 '1601',
 '348123',
 '349208',
 '374746',
 '248738',
 '364516',
 '345767',
 '345779',
 '330932',
 '113059',
 '14885',
 '3101278',
 '6608',
 '392086',
 '343275',
 '343276',
 '347466',
 '5734',
 '2315',
 '364500',
 '374910',
 '17754',
 '17759',
 '231919',
 '24436

In [79]:
train_df.Ticket=new_arr

In [80]:
train_df.Ticket

0        21171
1        17599
2      3101282
3       113803
4       373450
        ...   
886     211536
887     112053
888       6607
889     111369
890     370376
Name: Ticket, Length: 881, dtype: object